In [15]:
%autosave 60
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
from pathlib import Path

/home/gbiamby/anaconda3/envs/geoscreens/lib/python3.9/site-packages/PIL/Image.py:110: RuntimeWarning: The _imaging extension was built for another version of Pillow or PIL:
Core version: 8.4.0
Pillow version: 9.0.0
  warnings.warn(str(v), RuntimeWarning)
[autoreload of PIL.Image failed: Traceback (most recent call last):
  File "/home/gbiamby/anaconda3/envs/geoscreens/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/gbiamby/anaconda3/envs/geoscreens/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/home/gbiamby/anaconda3/envs/geoscreens/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 790, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_

Autosaving every 60 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
p = Path("../").resolve()

In [17]:
import json
import logging
import os
import pickle
import sys
from collections import Counter, OrderedDict
from copy import deepcopy
from datetime import datetime, timedelta
from io import BytesIO
from pathlib import Path
from types import ModuleType
from typing import Dict, List, Optional, Tuple, Union, cast

import cv2
import matplotlib as plt
import numpy as np
import pandas as pd
import PIL
import PIL.Image as pil_img
import seaborn as sns
import sklearn as skl
from IPython.display import Image, display
from matplotlib.patches import Rectangle
from matplotlib_inline.backend_inline import set_matplotlib_formats
from termcolor import colored
from tqdm.contrib import tenumerate, tmap, tzip
from tqdm.contrib.bells import tqdm, trange

In [18]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", 15)
pd.set_option("display.max_rows", 50)
# Suitable default display for floats
pd.options.display.float_format = "{:,.2f}".format
# matplotlib options
set_matplotlib_formats("pdf", "png")
plt.rcParams["savefig.dpi"] = 75
plt.rcParams["figure.autolayout"] = False
plt.rcParams["figure.figsize"] = 10, 6
plt.rcParams["axes.labelsize"] = 18
plt.rcParams["axes.titlesize"] = 20
plt.rcParams["font.size"] = 16
plt.rcParams["lines.linewidth"] = 2.0
plt.rcParams["lines.markersize"] = 8
plt.rcParams["legend.fontsize"] = 14
plt.rcParams["text.usetex"] = True
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "cm"
plt.rcParams["text.latex.preamble"] = "\\usepackage{subdepth}, \\usepackage{type1cm}"

# This one is optional -- change graphs to SVG only use if you don't have a
# lot of points/lines in your graphs. Can also just use ['retina'] if you
# don't want SVG.
%config InlineBackend.figure_formats = ["retina"]
set_matplotlib_formats("pdf", "png")

In [19]:
# VIDEO_PATH = Path("/shared/g-luo/geoguessr/videos").resolve()
# OUT_PATH = Path("/shared/gbiamby/geo/screenshots/screen_samples_auto").resolve()
# assert VIDEO_PATH.exists()
# assert OUT_PATH.exists()

## Game State Detection

### Load Detections

In [20]:
def parse_tuple(s: str):
    if isinstance(s, str):
        result = s.replace("(", "[").replace(")", "]")
        result = result.replace("'", '"').strip()
        result = result.replace(",]", "]")
        if result:
            # print(result)
            return tuple(sorted((json.loads(result))))
        else:
            return set()
    return s


def parse_dict(s: str):
    if isinstance(s, str):
        return json.loads(s.replace("'", '"'))
    return s


def load_detections_csv(
    video_id: str, split: str = "val", model: str = "geoscreens_009-resnest50_fpn-with_augs"
) -> pd.DataFrame:
    csv_path = Path(
        f"/shared/gbiamby/geo/segment/detections/{model}/{split}/df_frame_dets-video_id_{video_id}.csv",
    )
    df = pd.read_csv(csv_path)
    df.frame_id = df.frame_id.astype(int)
    df.label_ids = df.label_ids.apply(lambda x: parse_dict(x))
    df.labels = df.labels.apply(lambda x: parse_dict(x))
    df.labels_set = df.labels_set.apply(lambda x: parse_tuple(x))
    df.scores = df.scores.apply(lambda x: parse_dict(x))
    df.bboxes = df.bboxes.apply(lambda x: parse_dict(x))

    return df


def load_detections(
    video_id: str, split: str = "val", model: str = "geoscreens_009-resnest50_fpn-with_augs"
) -> pd.DataFrame:
    dets_path = Path(
        f"/shared/gbiamby/geo/segment/detections/{model}/{split}/df_frame_dets-video_id_{video_id}.csv",
    )
    if dets_path.suffix == ".csv":
        df = load_detections_csv(video_id, split=split, model=model)
    else:
        df = pickle.load(open(dets_path, "rb"))

    if "frame_time" not in df.columns:
        df["frame_time"] = df.apply(lambda x: f"{x.frame_id/4.0:04}", axis=1)
    if "seconds" not in df.columns:
        df["seconds"] = df.frame_id.apply(lambda frame_id: frame_id / 4.0)
    if "time" not in df.columns:
        df["time"] = df.frame_id.apply(
            lambda frame_id: datetime.utcfromtimestamp(frame_id / 4.0).strftime("%H:%M:%S:%f")
        )
    return df


# def get_categories(geoscreens_version: str = "009"):
#     data = json.load(
#         open(
#             f"/home/gbiamby/proj/geoscreens/datasets/geoscreens_{geoscreens_version}/geoscreens_{geoscreens_version}.json",
#             "r",
#         )
#     )
#     return data["categories"]

In [21]:
# # Show most common sets of UI elements that appear together in a single frame
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(pd.DataFrame(df_framedets.labels_set.value_counts()))

### Detect Game States for Each Frame

In [22]:
def apply_smoothing(
    df_framedets: pd.DataFrame, window_size: int = 5, direction: str = "backward"
) -> None:
    smoothed = []
    current_state = df_framedets.loc[0]["game_state"]
    direction = direction.replace("backwards", "backward").replace("forwards", "forward")
    if direction == "backward":
        buffer = [df_framedets.loc[0]["game_state"]] * window_size
        for i, row in df_framedets.iterrows():
            buffer.pop(0)
            buffer.append(row.game_state)
            counter = Counter(buffer)
            current_state = counter.most_common()[0]
            smoothed.append(current_state)
    else:
        # buffer = df_framedets.loc[:window_size]["game_state"].tolist()
        buffer = [df_framedets.loc[0]["game_state"]] * window_size
        print("Buffer: ", buffer)
        for i, row in df_framedets.iterrows():
            if 2860 <= i <= 2880:
                print(buffer)
            buffer.pop(0)
            buffer.append(
                df_framedets.loc[min(i + window_size, df_framedets.shape[0] - 1)].game_state
            )
            counter = Counter(buffer)
            current_state = counter.most_common()[0]
            smoothed.append(current_state)

    df_framedets["game_state_smoothed"] = [s[0] for s in smoothed]


def add_state_transition(state_transitions, row: pd.Series, from_state: str, to_state: str):
    state_transitions.extend(
        [
            {
                "state": from_state,
                "frame_id": row.frame_id,
                "end_frame_id": row.frame_id,
                "end_sec": row.seconds,
                "end_time": row.time,
            },
            {
                "state": to_state,
                "frame_id": row.frame_id,
                "start_frame_id": row.frame_id,
                "end_frame_id": None,
                "start_sec": row.seconds,
                "end_sec": None,
                "start_time": row.time,
                "end_time": None,
            },
        ]
    )


def get_segments(df_framedets: pd.DataFrame, smoothing=False) -> List[Dict]:
    current_state = "out_of_game"
    state_transitions = []
    state_key = "game_state_smoothed" if smoothing else "game_state"
    for i, row in df_framedets.iterrows():
        if current_state != "in_game" and row[state_key] == "in_game":
            add_state_transition(state_transitions, row, "out_of_game", "in_game")
            current_state = "in_game"
        elif current_state == "in_game":
            if row[state_key] == "in_game":
                continue
            else:
                add_state_transition(state_transitions, row, "in_game", "out_of_game")
                current_state = row[state_key]
        elif current_state == "end_of_game" and row[state_key] in [
            "out_of_game",
            "end_of_game",
            "unknown",
        ]:
            current_state = "end_of_game"

    return state_transitions


def get_oog_segments(segments: List[Dict], game_state: str = None):
    i = 1
    segs = []
    while i + 1 < len(segments):
        seg = segments[i]
        seg.update(segments[i + 1])
        seg["duration_sec"] = seg["end_sec"] - seg["start_sec"]
        seg["duration_hms"] = datetime.utcfromtimestamp(seg["duration_sec"]).strftime("%H:%M:%S:%f")

        if game_state and seg["state"] == game_state:
            segs.append(seg)
        elif not game_state:
            segs.append(seg)
        i += 2
    return segs


def style_correct(s, props=""):
    return np.where(s == 1, props, "")


def style_wrong(s, props=""):
    return np.where(s == 0, props, "")


def compare_to_gt(segs: List[Dict], gt: Dict):
    gt_oog_segs = gt["oog_segs"]
    for seg in segs:
        if seg["state"] == "out_of_game":
            is_correct = (
                len(
                    list(
                        filter(
                            lambda s: float(s[1]) - 2.0 < seg["end_sec"] <= float(s[1]) + 2.0,
                            gt_oog_segs,
                        )
                    )
                )
                == 1
            )
            seg["is_correct"] = is_correct

In [24]:
seg_gt = {
    "AF9uezxZDeE": {
        "in_game_segs_count": 5,
        "oog_segs": [(39, 141), (248, 275), (344, 352), (439, 440), (562, 650)],
    },
    "9RQUIk1OwAY": {
        "in_game_segs_count": 5,
        "oog_segs": [(185, 212), (394, 415), (597, 619), (801, 811), (994, 1022)],
    },
    "S5Ne5eoHxsY": {
        "in_game_segs_count": 5,
        "oog_segs": [(186, 212), (394, 405), (588, 608), (790, 809), (992, 1020)],
    },
    "nyHeQWnm8YA": {
        "in_game_segs_count": 5,
        "oog_segs": [(223, 231), (370, 377), (552, 554), (735, 742), (904, 929)],
    },
    "hZWt1PYH3hI": {
        "in_game_segs_count": 5,
        "oog_segs": [(117, 137), (314, 334), (471, 492), (660, 683), (866, 906)],
    },
    "dY1RXh-43q4": {
        "in_game_segs_count": 5,
        "oog_segs": [(158, 171), (287, 297), (418, 455), (577, 622), (740, 777)],
    },
    "83m9ys4kxro": {
        "in_game_segs_count": 5,
        "oog_segs": [(184, 206), (386, 398), (579, 592), (774, 806), (916, 951)],
    },
    "osTwgzWluVs": {
        "in_game_segs_count": 5,
        "oog_segs": [(147, 152), (272, 279), (360, 365), (525, 541), (671, 696)],
    },
    "o8qQAjkaXMM": {
        "in_game_segs_count": 20,
        "oog_segs": [
            (59, 67),
            (99, 107),
            (139, 147),
            (180, 195),
            (228, 262),
            (294, 299),
            (332, 346),
            (374, 377),
            (410, 425),
            (458, 482),
            (514, 528),
            (560, 566),
            (599, 607),
            (639, 647),
            (680, 708),
            (740, 757),
            (790, 793),
            (825.50, 838.50),
            (870.00, 886.75),
            # (825, 837),
            # (919, 937),
        ],
    },
}

In [38]:
game_state_ui_elements = OrderedDict(
    {
        "pre_game": {
            "any": [
                ["play", "left_menu_dark"],
                ["game_about_to_start_box_white", "left_menu_dark"],
                ["battle_royale_start_menu_w_logo"],
                ["game_about_to_start_box_white", "next_orange_btn"],
                ["setup_round_time_limit_box"],
            ],
            "exact": [],
        },
        "between_rounds": {
            "any": [
                ["left_menu_dark", "challenge_high_score_board"],
                ["play_next_round", "points_bar", "did_you_enjoy_this_location"],
                ["play_next_round", "points_bar_two_bars", "did_you_enjoy_this_location"],
                ["btw_rounds_points_bar_blue_wide", "play_next_round"],
            ],
            "exact": [
                ["in_game_mini_map", "status_bar", "points_bar", "game_title"],
                ["btw_rounds_points_bar_blue_wide"],
            ],
        },
        "between_round_or_game_ambiguous": {
            "any": [
                ["points_bar_two_bars", "did_you_enjoy_this_location", "status_bar"],
                ["between_rounds_box_white", "did_you_enjoy_this_location", "status_bar"],
                ["points_bar_two_bars", "status_bar"],
                ["points_bar_two_bars", "status_bar", "game_title"],
                ["points_bar_two_bars", "did_you_enjoy_this_location", "status_bar", "game_title"],
                ["points_bar_two_bars", "status_bar", "share_challenge_box_white"],
                ["points_bar_two_bars", "challenge_high_score_board", "status_bar"],
                ["points_bar_two_bars", "status_bar", "game_title", "share_challenge_box_white"],
                [
                    "between_rounds_box_white",
                    "did_you_enjoy_this_location",
                    "status_bar",
                    "game_title",
                ],
                ["points_bar", "left_menu_dark"],
                ["points_bar", "did_you_enjoy_this_location"],
                ["points_bar_two_bars", "did_you_enjoy_this_location"],
                ["status_bar", "points_bar", "game_title"],
                ["left_menu_dark"],
                ["left_menu_dark", "status_bar", "game_title"],
                ["left_menu_dark", "other"],
                ["left_menu_dark", "points_bar"],
                ["left_menu_dark", "points_bar_two_bars"],
            ],
            "exact": [],
        },
        "in_game": {
            "any": [
                # The ones with "in_game_mini_map" in first position will get auto-expanded:
                ["in_game_mini_map", "guess"],
                ["in_game_mini_map", "guess_grey"],
                ["in_game_mini_map", "make_a_guess"],
                ["in_game_mini_map", "place_your_pin_grey"],
                ["in_game_mini_map", "guess_w_icon_only"],
                # Won't get expanded:
                ["status_bar", "in_game_mini_map"],
                ["status_bar", "in_game_map_expanded"],
                ["status_bar_purple", "in_game_mini_map"],
                ["status_bar_purple", "in_game_map_expanded"],
            ],
            "exact": [
                ["status_bar_white", "status_bar", "game_title"],
                ["status_bar_white", "other", "status_bar", "game_title"],
                ["game_title", "status_bar_white", "make_a_guess_expanded", "status_bar"],
                ["status_bar_purple"],
            ],
        },
        "between_games": {
            "any": [
                ["left_menu_dark", "challenge_high_score_board"],
                ["points_bar", "show_high_score"],
                ["points_bar_two_bars", "show_high_score"],
                ["try_another_map", "points_bar_two_bars", "show_full_results"],
                ["points_bar_two_bars", "show_full_results"],
                ["high_score_box", "leader_board"],
            ],
            "exact": [],
        },
        "out_of_game": {
            "any": [
                ["google_mini_map"],
            ],
            "exact": [
                [],
                ["other"],
            ],
        },
        "unknown": {
            "any": [],
            "exact": [],
        },
    }
)

for state, match_types in game_state_ui_elements.items():
    for match_type, ui_element_combos in match_types.items():
        if match_type == "any" and state == "in_game":
            # expand the map / guess button combos to include all possible
            # expanded/not-expanded combinations for the in_game_map and the corresponding
            # "guess" button
            map_combos = [
                ui_combo for ui_combo in ui_element_combos if ui_combo[0] == "in_game_mini_map"
            ]
            for ui_combo in map_combos:
                guess_button_label = ui_combo[1]
                ui_element_combos.append(["in_game_mini_map", f"{guess_button_label}_expanded"])
                ui_element_combos.append(["in_game_map_expanded", guess_button_label])
                ui_element_combos.append(["in_game_map_expanded", f"{guess_button_label}_expanded"])

        # Convert the ui element lists to sets:
        match_types[match_type] = [set(elements) for elements in ui_element_combos]


def classify_frame(dets: dict) -> str:
    """
    Input is a row of a pd.DataFrame. The row contains object detector output
    for the geoguessr UI elements.
    """
    label_set = set(dets["labels_set"])
    if len(label_set) == 0:
        return "out_of_game"

    for state, match_types in game_state_ui_elements.items():
        for ui_combo in match_types["exact"]:
            if ui_combo == label_set:
                return state

        for ui_combo in match_types["any"]:
            intersection = ui_combo.intersection(label_set)
            if len(ui_combo) == len(intersection):
                return state

    return "unknown"


# game_state_ui_elements

---

## Debug

In [45]:
def apply_smoothing(
    df_framedets: pd.DataFrame, window_size: int = 5, direction: str = "forward"
) -> None:
    smoothed = []
    current_state = df_framedets.loc[0]["game_state"]
    direction = direction.replace("backwards", "backward").replace("forwards", "forward")
    prev_state = current_state
    if direction == "backward":
        buffer = [df_framedets.loc[0]["game_state"]] * window_size
        for i, row in df_framedets.iterrows():
            buffer.pop(0)
            buffer.append(row.game_state)
            counter = Counter(buffer)
            current_state = counter.most_common()[0]
            smoothed.append(current_state)
    else:
        buffer = [df_framedets.loc[0]["game_state"]] * window_size
        print("Buffer: ", buffer)
        for i, row in df_framedets.iterrows():
            buffer.pop(0)
            buffer.append(
                df_framedets.loc[min(i + window_size, df_framedets.shape[0] - 1)].game_state
            )
            if prev_state == row.game_state:
                smoothed.append((row.game_state, 0))
            else:
                counter = Counter(buffer)
                current_state = counter.most_common()[0]
                smoothed.append(current_state)
            prev_state = smoothed[-1][0]

    df_framedets["game_state_smoothed"] = [s[0] for s in smoothed]


video_id = "AF9uezxZDeE"
video_id = "nyHeQWnm8YA"
# gs_010_with_augs--geoscreens_010-model_faster_rcnn-bb_resnest50_fpn-8b23604566/val/df_frame_dets-video_id_AF9uezxZDeE
df_framedets = load_detections(
    video_id,
    split="val",
    ## This one gets (5 / 9 correct, smoothing=True), (3 / 9 correct, smoothing=False):
    # model="output/gs_010_with_augs--geoscreens_010-model_faster_rcnn-bb_resnest50_fpn-8b23604566",
    ### This one gets (6 / 9 correct, smoothing=True), ((6 / 9 correct, smoothing=False)):
    # model="output/gs_010_extra_augs--geoscreens_010-model_faster_rcnn-bb_resnest50_fpn-024f52f6dd",
    model="gs_011_extra_augs--geoscreens_011-model_faster_rcnn-bb_resnest50_fpn-3f36fb97fa"
)
df_framedets["game_state"] = df_framedets.apply(classify_frame, axis=1)
apply_smoothing(df_framedets, window_size=5, direction="forward")
seg = get_segments(df_framedets, smoothing=True)
segs_collapsed = get_oog_segments(seg)

#
print("video_id: ", video_id)
compare_to_gt(segs_collapsed, seg_gt[video_id])
df_seg = pd.DataFrame(segs_collapsed)
df_seg_styled = df_seg.style.apply(
    style_correct, props="color:white;background-color:green", axis=0, subset=["is_correct"]
).apply(
    style_wrong, props="color:white;background-color:red", axis=0, subset=["is_correct"]
)
display(df_seg_styled)
num_in_game_segments = df_seg[df_seg.state == "in_game"].shape[0]
print("num in_game: ", num_in_game_segments, ", gt: ", seg_gt[video_id])
if num_in_game_segments == seg_gt[video_id]["in_game_segs_count"]:
    print(colored("in_game segments count is: CORRECT!!!!!", color="green"))
else:
    print(colored("in_game segments count is: WRONG!!!", color="white", on_color="on_red"))

Buffer:  ['pre_game', 'pre_game', 'pre_game', 'pre_game', 'pre_game']
video_id:  nyHeQWnm8YA


,state,frame_id,start_frame_id,end_frame_id,start_sec,end_sec,start_time,end_time,duration_sec,duration_hms,is_correct
0,in_game,891,207,891,51.750000,222.750000,00:00:51:750000,00:03:42:750000,171.000000,00:02:51:000000,nan
1,out_of_game,929,891,929,222.750000,232.250000,00:03:42:750000,00:03:52:250000,9.500000,00:00:09:500000,1
2,in_game,1478,929,1478,232.250000,369.500000,00:03:52:250000,00:06:09:500000,137.250000,00:02:17:250000,nan
3,out_of_game,1510,1478,1510,369.500000,377.500000,00:06:09:500000,00:06:17:500000,8.000000,00:00:08:000000,1
4,in_game,2205,1510,2205,377.500000,551.250000,00:06:17:500000,00:09:11:250000,173.750000,00:02:53:750000,nan
5,out_of_game,2217,2205,2217,551.250000,554.250000,00:09:11:250000,00:09:14:250000,3.000000,00:00:03:000000,1
6,in_game,2943,2217,2943,554.250000,735.750000,00:09:14:250000,00:12:15:750000,181.500000,00:03:01:500000,nan
7,out_of_game,2971,2943,2971,735.750000,742.750000,00:12:15:750000,00:12:22:750000,7.000000,00:00:07:000000,1
8,in_game,3614,2971,3614,742.750000,903.500000,00:12:22:750000,00:15:03:500000,160.750000,00:02:40:750000,nan


num in_game:  5 , gt:  {'in_game_segs_count': 5, 'oog_segs': [(223, 231), (370, 377), (552, 554), (735, 742), (904, 929)]}
in_game segments count is: CORRECT!!!!!


### Total Frames Per Game States

In [40]:
print("Video_id: ", video_id)
pd.DataFrame(df_framedets.game_state.value_counts())

Video_id:  AF9uezxZDeE


,game_state
in_game,1918
between_round_or_game_ambiguous,787
between_rounds,172
pre_game,132
out_of_game,109
unknown,41


### Show ui combos that result in game_state "unknown":

In [41]:
print("Video_id: ", video_id)
pd.DataFrame(
    df_framedets[df_framedets.game_state == "unknown"]
    .groupby(["labels_set"])
    .agg(cnt=("frame_id", "count"))
).sort_values("cnt", ascending=False)

Video_id:  AF9uezxZDeE


,cnt
labels_set,
"(url,)",34
"(other, url)",6
"(game_title, status_bar, status_bar_white, url)",1


### Show all Frames For game_state==unknown 

In [42]:
print("Video_id: ", video_id)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df_framedets[df_framedets.game_state_smoothed == "unknown"][
            [
                "label_ids",
                "labels_set",
                # "scores",
                # "labels",
                # "bboxes",
                "label_set_count",
                "seconds",
                "time",
                "game_state",
                "game_state_smoothed",
            ]
        ]
    )

Video_id:  AF9uezxZDeE


,label_ids,labels_set,label_set_count,seconds,time,game_state,game_state_smoothed
2601,[68],"(url,)",623,650.25,00:10:50:250000,unknown,unknown
2602,[68],"(url,)",623,650.50,00:10:50:500000,unknown,unknown
2603,[68],"(url,)",623,650.75,00:10:50:750000,unknown,unknown
2604,[68],"(url,)",623,651.00,00:10:51:000000,unknown,unknown
2605,[68],"(url,)",623,651.25,00:10:51:250000,unknown,unknown
2606,[68],"(url,)",623,651.50,00:10:51:500000,unknown,unknown
2607,[68],"(url,)",623,651.75,00:10:51:750000,unknown,unknown
2614,"[43, 68]","(other, url)",623,653.50,00:10:53:500000,unknown,unknown
2615,[68],"(url,)",623,653.75,00:10:53:750000,unknown,unknown
2616,[68],"(url,)",623,654.00,00:10:54:000000,unknown,unknown


In [43]:
# from geoscreens.pseudolabels import reverse_point

# for box in boxes:
#     print(
#         reverse_point(box["xmin"], box["ymin"], 1280, 720, 640),
#         reverse_point(box["xmax"], box["ymax"], 1280, 720, 640),
#     )

In [44]:
print("Video_id: ", video_id)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df_framedets[(df_framedets.frame_id >= 3650) & (df_framedets.frame_id >= 3280)][
            # df_framedets[df_framedets.frame_id >= 0][
            # df_framedets[
            [
                "frame_id",
                "label_ids",
                "labels_set",
                # "scores",
                "labels",
                "bboxes",
                "label_set_count",
                "seconds",
                "time",
                "game_state",
                "game_state_smoothed",
            ]
        ]
    )

Video_id:  AF9uezxZDeE


,frame_id,label_ids,labels_set,labels,bboxes,label_set_count,seconds,time,game_state,game_state_smoothed


### Find frames that have a specific set of UI elements

In [519]:
print("Video_id: ", video_id)
df_framedets[df_framedets.labels_set == ("status_bar_white", "other", "status_bar", "game_title")]

,frame_id,label_ids,labels,labels_set,scores,bboxes,label_set_count,seconds,time,game_state,game_state_smoothed


---

## Segment many videos at once

In [384]:
segments = {}
val_ids = [
    "AF9uezxZDeE",
    "9RQUIk1OwAY",
    "S5Ne5eoHxsY",
    "nyHeQWnm8YA",
    "hZWt1PYH3hI",
    "dY1RXh-43q4",
    "83m9ys4kxro",
    "osTwgzWluVs",
    "o8qQAjkaXMM",
]
for video_id in val_ids:
    print("")
    print("video_id: ", video_id)
    df_framedets = load_detections(
        video_id,
        split="val",
        ## This one gets (5 / 9 correct, smoothing=True), (3 / 9 correct, smoothing=False):
        # model="output/gs_010_with_augs--geoscreens_010-model_faster_rcnn-bb_resnest50_fpn-8b23604566",
        ## This one gets (7 / 9 correct, smoothing=True), ((7 / 9 correct, smoothing=False)):
        model="output/gs_010_extra_augs--geoscreens_010-model_faster_rcnn-bb_resnest50_fpn-024f52f6dd",
    )
    # # This one gets (5 / 9 correct, smoothing=True), (3 / 9 correct, smoothing=False):
    # df_framedets = load_detections(video_id, split="val")
    df_framedets["game_state"] = df_framedets.apply(classify_frame, axis=1)
    apply_smoothing(df_framedets)
    seg = get_segments(df_framedets, smoothing=True)
    segments[video_id] = get_oog_segments(seg)


video_id:  AF9uezxZDeE
Buffer:  ['pre_game', 'pre_game', 'pre_game', 'pre_game', 'pre_game']

video_id:  9RQUIk1OwAY
Buffer:  ['out_of_game', 'out_of_game', 'out_of_game', 'out_of_game', 'out_of_game']

video_id:  S5Ne5eoHxsY
Buffer:  ['out_of_game', 'out_of_game', 'out_of_game', 'out_of_game', 'out_of_game']

video_id:  nyHeQWnm8YA
Buffer:  ['pre_game', 'pre_game', 'pre_game', 'pre_game', 'pre_game']

video_id:  hZWt1PYH3hI
Buffer:  ['out_of_game', 'out_of_game', 'out_of_game', 'out_of_game', 'out_of_game']

video_id:  dY1RXh-43q4
Buffer:  ['pre_game', 'pre_game', 'pre_game', 'pre_game', 'pre_game']

video_id:  83m9ys4kxro
Buffer:  ['out_of_game', 'out_of_game', 'out_of_game', 'out_of_game', 'out_of_game']

video_id:  osTwgzWluVs
Buffer:  ['out_of_game', 'out_of_game', 'out_of_game', 'out_of_game', 'out_of_game']

video_id:  o8qQAjkaXMM
Buffer:  ['pre_game', 'pre_game', 'pre_game', 'pre_game', 'pre_game']


In [408]:
for video_id, seg in segments.items():
    print("")
    print("")
    print("=" * 120)
    print("video_id: ", video_id)
    compare_to_gt(seg, seg_gt[video_id])
    df_seg = pd.DataFrame(seg)
    df_seg_styled = df_seg.style.apply(
        style_correct, props="color:white;background-color:green", axis=0, subset=["is_correct"]
    ).apply(
        style_wrong, props="color:white;background-color:red", axis=0, subset=["is_correct"]
    )
    display(df_seg_styled)
    num_in_game_segments = df_seg[df_seg.state == "in_game"].shape[0]
    print("num in_game: ", num_in_game_segments, ", gt: ", seg_gt[video_id]["in_game_segs_count"])
    if num_in_game_segments == seg_gt[video_id]["in_game_segs_count"]:
        print(colored("in_game segments count is: CORRECT!!!!!", color="green"))
    else:
        print(colored("in_game segments count is: WRONG!!!", color="red", on_color="on_yellow"))



video_id:  AF9uezxZDeE


,state,frame_id,start_frame_id,end_frame_id,start_sec,end_sec,start_time,end_time,duration_sec,duration_hms,is_correct
0,in_game,554,158,554,39.500000,138.500000,00:00:39:500000,00:02:18:500000,99.000000,00:01:39:000000,nan
1,out_of_game,567,554,567,138.500000,141.750000,00:02:18:500000,00:02:21:750000,3.250000,00:00:03:250000,1
2,in_game,992,567,992,141.750000,248.000000,00:02:21:750000,00:04:08:000000,106.250000,00:01:46:250000,nan
3,out_of_game,1104,992,1104,248.000000,276.000000,00:04:08:000000,00:04:36:000000,28.000000,00:00:28:000000,1
4,in_game,1373,1104,1373,276.000000,343.250000,00:04:36:000000,00:05:43:250000,67.250000,00:01:07:250000,nan
5,out_of_game,1409,1373,1409,343.250000,352.250000,00:05:43:250000,00:05:52:250000,9.000000,00:00:09:000000,1
6,in_game,1754,1409,1754,352.250000,438.500000,00:05:52:250000,00:07:18:500000,86.250000,00:01:26:250000,nan
7,out_of_game,1762,1754,1762,438.500000,440.500000,00:07:18:500000,00:07:20:500000,2.000000,00:00:02:000000,1
8,in_game,2246,1762,2246,440.500000,561.500000,00:07:20:500000,00:09:21:500000,121.000000,00:02:01:000000,nan
9,out_of_game,2637,2246,2637,561.500000,659.250000,00:09:21:500000,00:10:59:250000,97.750000,00:01:37:750000,0


num in_game:  8 , gt:  5
in_game segments count is: WRONG!!!


video_id:  9RQUIk1OwAY


,state,frame_id,start_frame_id,end_frame_id,start_sec,end_sec,start_time,end_time,duration_sec,duration_hms,is_correct
0,in_game,741,31,741,7.750000,185.250000,00:00:07:750000,00:03:05:250000,177.500000,00:02:57:500000,nan
1,out_of_game,850,741,850,185.250000,212.500000,00:03:05:250000,00:03:32:500000,27.250000,00:00:27:250000,1
2,in_game,1576,850,1576,212.500000,394.000000,00:03:32:500000,00:06:34:000000,181.500000,00:03:01:500000,nan
3,out_of_game,1663,1576,1663,394.000000,415.750000,00:06:34:000000,00:06:55:750000,21.750000,00:00:21:750000,1
4,in_game,2388,1663,2388,415.750000,597.000000,00:06:55:750000,00:09:57:000000,181.250000,00:03:01:250000,nan
5,out_of_game,2477,2388,2477,597.000000,619.250000,00:09:57:000000,00:10:19:250000,22.250000,00:00:22:250000,1
6,in_game,3203,2477,3203,619.250000,800.750000,00:10:19:250000,00:13:20:750000,181.500000,00:03:01:500000,nan
7,out_of_game,3246,3203,3246,800.750000,811.500000,00:13:20:750000,00:13:31:500000,10.750000,00:00:10:750000,1
8,in_game,3975,3246,3975,811.500000,993.750000,00:13:31:500000,00:16:33:750000,182.250000,00:03:02:250000,nan


num in_game:  5 , gt:  5
in_game segments count is: CORRECT!!!!!


video_id:  S5Ne5eoHxsY


...

num in_game:  5 , gt:  5
in_game segments count is: CORRECT!!!!!


video_id:  osTwgzWluVs


,state,frame_id,start_frame_id,end_frame_id,start_sec,end_sec,start_time,end_time,duration_sec,duration_hms,is_correct
0,in_game,585,31,585,7.750000,146.250000,00:00:07:750000,00:02:26:250000,138.500000,00:02:18:500000,nan
1,out_of_game,611,585,611,146.250000,152.750000,00:02:26:250000,00:02:32:750000,6.500000,00:00:06:500000,1
2,in_game,1086,611,1086,152.750000,271.500000,00:02:32:750000,00:04:31:500000,118.750000,00:01:58:750000,nan
3,out_of_game,1117,1086,1117,271.500000,279.250000,00:04:31:500000,00:04:39:250000,7.750000,00:00:07:750000,1
4,in_game,1439,1117,1439,279.250000,359.750000,00:04:39:250000,00:05:59:750000,80.500000,00:01:20:500000,nan
5,out_of_game,1462,1439,1462,359.750000,365.500000,00:05:59:750000,00:06:05:500000,5.750000,00:00:05:750000,1
6,in_game,2100,1462,2100,365.500000,525.000000,00:06:05:500000,00:08:45:000000,159.500000,00:02:39:500000,nan
7,out_of_game,2165,2100,2165,525.000000,541.250000,00:08:45:000000,00:09:01:250000,16.250000,00:00:16:250000,1
8,in_game,2684,2165,2684,541.250000,671.000000,00:09:01:250000,00:11:11:000000,129.750000,00:02:09:750000,nan


num in_game:  5 , gt:  5
in_game segments count is: CORRECT!!!!!


video_id:  o8qQAjkaXMM


,state,frame_id,start_frame_id,end_frame_id,start_sec,end_sec,start_time,end_time,duration_sec,duration_hms,is_correct
0,in_game,236,110,236,27.500000,59.000000,00:00:27:500000,00:00:59:000000,31.500000,00:00:31:500000,nan
1,out_of_game,269,236,269,59.000000,67.250000,00:00:59:000000,00:01:07:250000,8.250000,00:00:08:250000,1
2,in_game,394,269,394,67.250000,98.500000,00:01:07:250000,00:01:38:500000,31.250000,00:00:31:250000,nan
3,out_of_game,430,394,430,98.500000,107.500000,00:01:38:500000,00:01:47:500000,9.000000,00:00:09:000000,1
4,in_game,555,430,555,107.500000,138.750000,00:01:47:500000,00:02:18:750000,31.250000,00:00:31:250000,nan
5,out_of_game,591,555,591,138.750000,147.750000,00:02:18:750000,00:02:27:750000,9.000000,00:00:09:000000,1
6,in_game,717,591,717,147.750000,179.250000,00:02:27:750000,00:02:59:250000,31.500000,00:00:31:500000,nan
7,out_of_game,783,717,783,179.250000,195.750000,00:02:59:250000,00:03:15:750000,16.500000,00:00:16:500000,1
8,in_game,912,783,912,195.750000,228.000000,00:03:15:750000,00:03:48:000000,32.250000,00:00:32:250000,nan
9,out_of_game,1050,912,1050,228.000000,262.500000,00:03:48:000000,00:04:22:500000,34.500000,00:00:34:500000,1


num in_game:  20 , gt:  20
in_game segments count is: CORRECT!!!!!


In [343]:
for video_id, seg in segments.items():
    print("")
    print("video_id: ", video_id)
    # display(seg)
    for s in seg:
        print(s)


video_id:  AF9uezxZDeE
{'state': 'in_game', 'frame_id': 554, 'start_frame_id': 158, 'end_frame_id': 554, 'start_sec': 39.5, 'end_sec': 138.5, 'start_time': '00:00:39:500000', 'end_time': '00:02:18:500000', 'duration_sec': 99.0, 'duration_hms': '00:01:39:000000'}
{'state': 'out_of_game', 'frame_id': 567, 'start_frame_id': 554, 'end_frame_id': 567, 'start_sec': 138.5, 'end_sec': 141.75, 'start_time': '00:02:18:500000', 'end_time': '00:02:21:750000', 'duration_sec': 3.25, 'duration_hms': '00:00:03:250000', 'is_correct': True}
{'state': 'in_game', 'frame_id': 992, 'start_frame_id': 567, 'end_frame_id': 992, 'start_sec': 141.75, 'end_sec': 248.0, 'start_time': '00:02:21:750000', 'end_time': '00:04:08:000000', 'duration_sec': 106.25, 'duration_hms': '00:01:46:250000'}
{'state': 'out_of_game', 'frame_id': 1104, 'start_frame_id': 992, 'end_frame_id': 1104, 'start_sec': 248.0, 'end_sec': 276.0, 'start_time': '00:04:08:000000', 'end_time': '00:04:36:000000', 'duration_sec': 28.0, 'duration_hms'

### Show the game states sequence:

In [339]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df_framedets[df_framedets.seconds.isin([702.25, 702.5])][
            ["labels_set", "game_state", "time", "seconds"]
        ]
    )
    # display(df_framedets[["labels_set", "game_state", "time", "seconds"]])

,labels_set,game_state,time,seconds
2809,"(in_game_map_expanded,)",unknown,00:11:42:250000,702.25
2810,"(in_game_map_expanded,)",unknown,00:11:42:500000,702.50


In [262]:
df_framedets.game_state.unique()

array(['pre_game', 'between_round_or_game_ambiguous', 'unknown', 'between_games', 'in_game', 'between_rounds', 'out_of_game'], dtype=object)

,game_state
0,"(pre_game, 4)"
1,"(pre_game, 4)"
2,"(pre_game, 4)"
3,"(pre_game, 4)"
4,"(pre_game, 4)"
5,"(pre_game, 4)"
6,"(pre_game, 4)"
7,"(pre_game, 4)"
8,"(pre_game, 4)"
9,"(pre_game, 4)"


In [266]:
current_state = "out_of_game"
state_transitions = []
for i, row in df_framedets.iterrows():
    if current_state != "in_game" and row.game_state == "in_game":
        state_transitions.extend(
            [
                {
                    "state": "out_of_game",
                    "end_frame_id": row.frame_id,
                    "end_frame_time": row.seconds,
                },
                {
                    "state": "in_game",
                    "start_frame_id": row.frame_id,
                    "start_frame_time": row.seconds,
                },
            ]
        )
        current_state = "in_game"
    elif current_state == "in_game":
        if row.game_state == "in_game":
            continue
        else:
            state_transitions.extend(
                [
                    {
                        "state": "in_game",
                        "end_frame_id": row.frame_id,
                        "end_frame_time": row.seconds,
                    },
                    {
                        "state": "out_of_game",
                        "start_frame_id": row.frame_id,
                        "start_frame_time": row.seconds,
                    },
                ]
            )
            current_state = row.game_state

state_transitions

[{'state': 'out_of_game', 'end_frame_id': 158, 'end_frame_time': 39.5},
 {'state': 'in_game', 'start_frame_id': 158, 'start_frame_time': 39.5},
 {'state': 'in_game', 'end_frame_id': 554, 'end_frame_time': 138.5},
 {'state': 'out_of_game', 'start_frame_id': 554, 'start_frame_time': 138.5},
 {'state': 'out_of_game', 'end_frame_id': 567, 'end_frame_time': 141.75},
 {'state': 'in_game', 'start_frame_id': 567, 'start_frame_time': 141.75},
 {'state': 'in_game', 'end_frame_id': 992, 'end_frame_time': 248.0},
 {'state': 'out_of_game', 'start_frame_id': 992, 'start_frame_time': 248.0},
 {'state': 'out_of_game', 'end_frame_id': 1104, 'end_frame_time': 276.0},
 {'state': 'in_game', 'start_frame_id': 1104, 'start_frame_time': 276.0},
 {'state': 'in_game', 'end_frame_id': 1373, 'end_frame_time': 343.25},
 {'state': 'out_of_game', 'start_frame_id': 1373, 'start_frame_time': 343.25},
 {'state': 'out_of_game', 'end_frame_id': 1409, 'end_frame_time': 352.25},
 {'state': 'in_game', 'start_frame_id': 140

## Scratch / Junk

---


In [157]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df_framedets.loc[230:490][
            [
                "frame_id",
                "seconds",
                "labels",
                "labels_set",
                "game_state",
                "scores",
            ]
        ]
    )

,frame_id,seconds,labels,labels_set,game_state,scores
230,230,57.50,"[between_rounds_box_white, game_about_to_start_box_white, game_title, left_menu_dark, status_bar]","(game_about_to_start_box_white, game_title, between_rounds_box_white, left_menu_dark, status_bar)",pre_game,"[0.6099945306777954, 0.7488806247711182, 0.9509423971176147, 0.9944057464599609, 0.9975166320800781]"
231,231,57.75,"[between_rounds_box_white, game_about_to_start_box_white, game_title, left_menu_dark, play, status_bar]","(game_about_to_start_box_white, game_title, play, between_rounds_box_white, left_menu_dark, status_bar)",pre_game,"[0.6126360297203064, 0.7485108971595764, 0.9509128332138062, 0.9944043159484863, 0.5438895225524902, 0.9975181818008423]"
232,232,58.00,"[between_rounds_box_white, game_about_to_start_box_white, game_title, left_menu_dark, play, status_bar]","(game_about_to_start_box_white, game_title, play, between_rounds_box_white, left_menu_dark, status_bar)",pre_game,"[0.6105313301086426, 0.7475910186767578, 0.9509725570678711, 0.9944033622741699, 0.5533502101898193, 0.9975177049636841]"
233,233,58.25,"[between_rounds_box_white, game_about_to_start_box_white, game_title, left_menu_dark, play, status_bar]","(game_about_to_start_box_white, game_title, play, between_rounds_box_white, left_menu_dark, status_bar)",pre_game,"[0.6139984130859375, 0.7493197917938232, 0.9507407546043396, 0.9944045543670654, 0.5803712010383606, 0.9975194334983826]"
234,234,58.50,"[between_rounds_box_white, game_about_to_start_box_white, game_title, left_menu_dark, show_high_score, status_bar, status_bar_white]","(game_about_to_start_box_white, status_bar_white, game_title, between_rounds_box_white, left_menu_dark, show_high_score, status_bar)",pre_game,"[0.5244572162628174, 0.5186703205108643, 0.9450609087944031, 0.9945268034934998, 0.5099112391471863, 0.9973294734954834, 0.6404302716255188]"
235,235,58.75,"[between_rounds_box_white, game_about_to_start_box_white, game_title, left_menu_dark, status_bar, status_bar_white]","(game_about_to_start_box_white, status_bar_white, game_title, between_rounds_box_white, left_menu_dark, status_bar)",pre_game,"[0.5032690167427063, 0.51612788438797, 0.9544402956962585, 0.9945304989814758, 0.9973207116127014, 0.6780527830123901]"
236,236,59.00,"[game_about_to_start_box_white, game_title, left_menu_dark, status_bar, status_bar_white]","(game_about_to_start_box_white, status_bar_white, game_title, left_menu_dark, status_bar)",pre_game,"[0.5119966268539429, 0.954524576663971, 0.9945363998413086, 0.9973201155662537, 0.7745059132575989]"
237,237,59.25,"[game_about_to_start_box_white, game_title, left_menu_dark, status_bar, status_bar_white]","(game_about_to_start_box_white, status_bar_white, game_title, left_menu_dark, status_bar)",pre_game,"[0.542531430721283, 0.954529881477356, 0.9945346117019653, 0.9973158240318298, 0.6737319231033325]"
238,238,59.50,"[game_title, left_menu_dark, status_bar, status_bar_white]","(status_bar_white, left_menu_dark, status_bar, game_title)",between_round_or_game_ambiguous,"[0.954569399356842, 0.9945336580276489, 0.9973166584968567, 0.6803613305091858]"
239,239,59.75,"[game_title, left_menu_dark, show_high_score, status_bar]","(status_bar, left_menu_dark, show_high_score, game_title)",between_round_or_game_ambiguous,"[0.9524386525154114, 0.9945341348648071, 0.5063733458518982, 0.9973245859146118]"


In [164]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(df_framedets.tail(100))

,frame_id,label_ids,labels,labels_set,scores,bboxes,ingame_trio,label_set_count,game_state,seconds
6228,6228,"[17, 46, 52, 55, 57]","[game_title, points_bar, show_high_score, status_bar, status_bar_white]","(status_bar_white, points_bar, game_title, show_high_score, status_bar)","[0.9267572164535522, 0.9572892785072327, 0.8997751474380493, 0.9970371723175049, 0.5993921160697937]","[{'xmin': 9.228679656982422, 'ymin': 171.5681610107422, 'xmax': 93.92555236816406, 'ymax': 193.0092010498047}, {'xmin': 170.40907287597656, 'ymin': 306.2313232421875, 'xmax': 462.651611328125, 'ymax': 339.39080810546875}, {'xmin': 271.53546142578125, 'ymin': 349.8526611328125, 'xmax': 353.759033203125, 'ymax': 366.04351806640625}, {'xmin': 436.4908142089844, 'ymin': 171.96588134765625, 'xmax': 634.7816162109375, 'ymax': 194.07281494140625}, {'xmin': 184.99974060058594, 'ymin': 301.8109436035156, 'xmax': 458.1788330078125, 'ymax': 333.7883605957031}]",False,146,between_games,"1,557.00"
6229,6229,"[17, 46, 52, 55, 57]","[game_title, points_bar, show_high_score, status_bar, status_bar_white]","(status_bar_white, points_bar, game_title, show_high_score, status_bar)","[0.9267572164535522, 0.9572892785072327, 0.8997751474380493, 0.9970371723175049, 0.5993921160697937]","[{'xmin': 9.228679656982422, 'ymin': 171.5681610107422, 'xmax': 93.92555236816406, 'ymax': 193.0092010498047}, {'xmin': 170.40907287597656, 'ymin': 306.2313232421875, 'xmax': 462.651611328125, 'ymax': 339.39080810546875}, {'xmin': 271.53546142578125, 'ymin': 349.8526611328125, 'xmax': 353.759033203125, 'ymax': 366.04351806640625}, {'xmin': 436.4908142089844, 'ymin': 171.96588134765625, 'xmax': 634.7816162109375, 'ymax': 194.07281494140625}, {'xmin': 184.99974060058594, 'ymin': 301.8109436035156, 'xmax': 458.1788330078125, 'ymax': 333.7883605957031}]",False,146,between_games,"1,557.25"
6230,6230,"[17, 46, 52, 55, 57]","[game_title, points_bar, show_high_score, status_bar, status_bar_white]","(status_bar_white, points_bar, game_title, show_high_score, status_bar)","[0.9267482161521912, 0.9572681784629822, 0.899724006652832, 0.9970404505729675, 0.5994471907615662]","[{'xmin': 9.229270935058594, 'ymin': 171.5682373046875, 'xmax': 93.92586517333984, 'ymax': 193.00918579101562}, {'xmin': 170.40855407714844, 'ymin': 306.2311706542969, 'xmax': 462.65185546875, 'ymax': 339.3906555175781}, {'xmin': 271.53436279296875, 'ymin': 349.85321044921875, 'xmax': 353.75823974609375, 'ymax': 366.04388427734375}, {'xmin': 436.4678955078125, 'ymin': 171.96607971191406, 'xmax': 634.7818603515625, 'ymax': 194.07286071777344}, {'xmin': 185.00119018554688, 'ymin': 301.81103515625, 'xmax': 458.1789855957031, 'ymax': 333.7886962890625}]",False,146,between_games,"1,557.50"
6231,6231,"[17, 46, 52, 55, 57]","[game_title, points_bar, show_high_score, status_bar, status_bar_white]","(status_bar_white, points_bar, game_title, show_high_score, status_bar)","[0.9267482161521912, 0.9572681784629822, 0.899724006652832, 0.9970404505729675, 0.5994471907615662]","[{'xmin': 9.229270935058594, 'ymin': 171.5682373046875, 'xmax': 93.92586517333984, 'ymax': 193.00918579101562}, {'xmin': 170.40855407714844, 'ymin': 306.2311706542969, 'xmax': 462.65185546875, 'ymax': 339.3906555175781}, {'xmin': 271.53436279296875, 'ymin': 349.85321044921875, 'xmax': 353.75823974609375, 'ymax': 366.04388427734375}, {'xmin': 436.4678955078125, 'ymin': 171.96607971191406, 'xmax': 634.7818603515625, 'ymax': 194.07286071777344}, {'xmin': 185.00119018554688, 'ymin': 301.81103515625, 'xmax': 458.1789855957031, 'ymax': 333.7886962890625}]",False,146,between_games,"1,557.75"
6232,6232,"[17, 46, 52, 55, 57]","[game_title, points_bar, show_high_score, status_bar, status_bar_white]","(status_bar_white, points_bar, game_title, show_high_score, status_bar)","[0.9267572164535522, 0.9572892785072327, 0.8997751474380493, 0.9970371723175049, 0.5993921160697937]","[{'xmin': 9.228679656982422, 'ymin': 171.5681610107422, 'xmax': 93.92555236816406, 'ymax': 193.0092010498047},

---